# &AElig;co
***Calc module***
<br/><br/>

✈️♻️

This code is part of the Master's Thesis "A Life Cycle Approach for Aircraft Ecodesign", by Giácomo Parolin.

``Please enter the following information:``

In [1]:
import os
os.chdir("C:/Users/giparoli/Documents/Projetos/AEco")

input_path = './Data/Iris_inputs.xlsx'
input_sheet = 'Iris'
output_path = './Outputs/'+ input_sheet + '_outputs'
database_path = './Data/database_Iris.xlsx'

iterations = 1000
chunks = 'auto'  #'auto' or more than 500

In [2]:
from Tools_Iris import *
from Model_Iris import *

# LCI

The aircraft's life cycle was divided into four parts:
1. Development and Engineering
2. Manufacturing and Assembly
3. Operation
4. End-of-Life

First, we execute function to read inputs from designated `input_path` and transform it into Parameters and ParameterSet object `p`.

In [3]:
inputs = read_inputs(input_path, input_sheet)

In [4]:
p = ParameterSet(inputs, iterations, chunks)

Samples are stored in a xr.Dataset object accessible throught the `p.data` ParameterSet attribute.

Then, we run the function that calculates the functional unit for the study.

In [5]:
p = func_unit(p)

In [6]:
p.data["ha_life"].mean()

<xarray.DataArray 'ha_life' ()>
dask.array<mean_agg-aggregate, shape=(), dtype=float64, chunksize=(), chunktype=numpy.ndarray>

In [7]:
p.data

<xarray.Dataset>
Dimensions:             (i: 100000)
Coordinates:
  * i                   (i) int32 0 1 2 3 4 5 ... 99995 99996 99997 99998 99999
Data variables: (12/91)
    takt_US             (i) float64 dask.array<chunksize=(125,), meta=np.ndarray>
    takt_BR             (i) float64 dask.array<chunksize=(125,), meta=np.ndarray>
    OEW                 (i) float64 dask.array<chunksize=(125,), meta=np.ndarray>
    flights_year        (i) float64 dask.array<chunksize=(125,), meta=np.ndarray>
    FH                  (i) float64 dask.array<chunksize=(125,), meta=np.ndarray>
    productivity        (i) float64 dask.array<chunksize=(125,), meta=np.ndarray>
    ...                  ...
    ha_flight           (i) float64 dask.array<chunksize=(125,), meta=np.ndarray>
    ha_year             (i) float64 dask.array<chunksize=(125,), meta=np.ndarray>
    ha_life             (i) float64 dask.array<chunksize=(125,), meta=np.ndarray>
    ha_fleet            (i) float64 dask.array<chunksize=(125,), meta=np.ndarray>
    ha_fleet_US         (i) float64 dask.array<chunksize=(125,), meta=np.ndarray>
    ha_fleet_BR         (i) float64 dask.array<chunksize=(125,), meta=np.ndarray>

<br></br>
Executing the UP reading and writing functions.

In [8]:
UP_dataframe = read_unit_processes(database_path)

In [9]:
UP = unit_process_dataset(UP_dataframe)

Building the empty LCI to start the calculation.

In [10]:
inventory = LCI(name=input_sheet, iterations=iterations, UP=UP, parameters=p)

In [11]:
inventory.run()

<xarray.Dataset>
Dimensions:         (Substances: 1835, i: 100000)
Coordinates:
    Units           (Substances) object 'kg' 'kg' 'kg' 'kg' ... 'kg' 'kBq' 'kBq'
  * Substances      (Substances) MultiIndex
  - name            (Substances) object '1,4-Butanediol' ... 'Zirconium-95'
  - compartment     (Substances) object 'Water' 'Air' 'Air' ... 'Water' 'Air'
  - subcompartment  (Substances) object 'surface water' ... 'non-urban air or...
  * i               (i) int32 0 1 2 3 4 5 ... 99995 99996 99997 99998 99999
Data variables: (12/16)
    Office          (Substances, i) float64 dask.array<chunksize=(1835, 125), meta=np.ndarray>
    Infrastructure  (Substances, i) float64 dask.array<chunksize=(1835, 125), meta=np.ndarray>
    Capital         (Substances, i) float64 dask.array<chunksize=(1835, 125), meta=np.ndarray>
    Materials       (Substances, i) float64 dask.array<chunksize=(1835, 125), meta=np.ndarray>
    Factory         (Substances, i) float64 dask.array<chunksize=(1835, 125), meta=np.ndarray>
    Logistics       (Substances, i) float64 dask.array<chunksize=(1835, 125), meta=np.ndarray>
    ...              ...
    Fuel            (Substances, i) float64 dask.array<chunksize=(1835, 125), meta=np.ndarray>
    Recycling       (Substances, i) float64 dask.array<chunksize=(1835, 125), meta=np.ndarray>
    Incineration    (Substances, i) float64 dask.array<chunksize=(1835, 125), meta=np.ndarray>
    Landfill        (Substances, i) float64 dask.array<chunksize=(1835, 125), meta=np.ndarray>
    Prototypes      (Substances, i) float64 dask.array<chunksize=(1835, 125), meta=np.ndarray>
    Certification   (Substances, i) float64 dask.array<chunksize=(1835, 125), meta=np.ndarray>
Attributes:
    Name:     Antera_spray

The LCI data can be accessed by the `LCI.data` attribute.

# LCIA

Extracting the characterization factors from the excel file located in `database_path`.

In [12]:
MP_data, EP_data = read_CF(database_path)

Creating an instance of the characterization factors class:

In [13]:
CF = CharactFactors(MP_data, EP_data, UP.Substances)

Adjusting the NLT category CFs.

In [14]:
CF.MP.dataset['NLT'].data = np.negative(CF.MP.dataset['NLT'].data)
CF.MP.to_array();

The impacts CFs can be accessed by the `CharactFactors.Midpoint.dataset` or `CharactFactors.Endpoint.dataset` attributes.

In [15]:
display(CF.MP.dataset)
display(CF.EP.dataset)

<xarray.Dataset>
Dimensions:         (Substances: 1835)
Coordinates:
  * Substances      (Substances) MultiIndex
  - name            (Substances) object '1,4-Butanediol' ... 'Zirconium-95'
  - compartment     (Substances) object 'Water' 'Air' 'Air' ... 'Water' 'Air'
  - subcompartment  (Substances) object 'surface water' ... 'non-urban air or...
Data variables: (12/18)
    FE              (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    MRD             (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    POF             (Substances) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    MET             (Substances) float64 0.0 0.0 0.000283 ... 0.0 0.0 0.0
    TA              (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    ULO             (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    ...              ...
    ALO             (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    HT              (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    IR              (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    FD              (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    WD              (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    CC              (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0

<xarray.Dataset>
Dimensions:     (Categories: 18)
Coordinates:
  * Categories  (Categories) object 'CC' 'OD' 'TA' 'FE' ... 'WD' 'MRD' 'FD'
Data variables:
    HH          (Categories) float64 1.4e-06 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    ED          (Categories) float64 7.93e-09 0.0 5.8e-09 ... 0.0 0.0 0.0
    RA          (Categories) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0715 0.165

<br></br>
**Executing the Impact Assessment.**

In [16]:
aeco = LCIA.build(inventory, CF)

## Midpoint Results

Raw midpoint results can be accessed by the `LCIA.MP` attribute. Mean midpoint results are calculated by the `LCIA.mean` method, with pathway set to "MP".

In [17]:
display(aeco.MP)

<xarray.Dataset>
Dimensions:         (Categories: 18, i: 100000)
Coordinates:
  * i               (i) int32 0 1 2 3 4 5 ... 99995 99996 99997 99998 99999
  * Categories      (Categories) <U3 'FE' 'MRD' 'POF' 'MET' ... 'FD' 'WD' 'CC'
Data variables: (12/16)
    Office          (i, Categories) float64 dask.array<chunksize=(125, 18), meta=np.ndarray>
    Infrastructure  (i, Categories) float64 dask.array<chunksize=(125, 18), meta=np.ndarray>
    Capital         (i, Categories) float64 dask.array<chunksize=(125, 18), meta=np.ndarray>
    Materials       (i, Categories) float64 dask.array<chunksize=(125, 18), meta=np.ndarray>
    Factory         (i, Categories) float64 dask.array<chunksize=(125, 18), meta=np.ndarray>
    Logistics       (i, Categories) float64 dask.array<chunksize=(125, 18), meta=np.ndarray>
    ...              ...
    Fuel            (i, Categories) float64 dask.array<chunksize=(125, 18), meta=np.ndarray>
    Recycling       (i, Categories) float64 dask.array<chunksize=(125, 18), meta=np.ndarray>
    Incineration    (i, Categories) float64 dask.array<chunksize=(125, 18), meta=np.ndarray>
    Landfill        (i, Categories) float64 dask.array<chunksize=(125, 18), meta=np.ndarray>
    Prototypes      (i, Categories) float64 dask.array<chunksize=(125, 18), meta=np.ndarray>
    Certification   (i, Categories) float64 dask.array<chunksize=(125, 18), meta=np.ndarray>
Attributes:
    Name:     Antera_spray

In [18]:
aeco.mean("MP", by='sum').compute()

<xarray.DataArray 'stack-95e20ce77192a43fa9677e1b4c3eabbe' (Categories: 18)>
array([2.68881011e-02, 4.63544614e+01, 1.95520629e+00, 1.22134171e+01,
       1.56977892e+00, 8.40417765e-01, 5.76294172e-01, 3.59777448e+00,
       6.77128945e+00, 4.46009957e-03, 6.29668290e-03, 8.28722473e-01,
       8.52660639e+00, 7.52546032e+04, 4.49719641e+01, 2.57526651e+02,
       2.69693493e+02, 7.94042918e+02])
Coordinates:
  * Categories  (Categories) <U3 'FE' 'MRD' 'POF' 'MET' ... 'IR' 'FD' 'WD' 'CC'

## Endpoint Results

Raw endpoint results can be accessed by the `LCIA.EP` attribute. Mean endpoint results are calculated by the `LCIA.mean` method, with pathway set to "EP".

In [20]:
display(aeco.EP)

<xarray.Dataset>
Dimensions:         (AOP: 3, i: 100000)
Coordinates:
  * i               (i) int32 0 1 2 3 4 5 ... 99995 99996 99997 99998 99999
  * AOP             (AOP) <U2 'HH' 'ED' 'RA'
Data variables: (12/16)
    Office          (i, AOP) float64 dask.array<chunksize=(125, 3), meta=np.ndarray>
    Infrastructure  (i, AOP) float64 dask.array<chunksize=(125, 3), meta=np.ndarray>
    Capital         (i, AOP) float64 dask.array<chunksize=(125, 3), meta=np.ndarray>
    Materials       (i, AOP) float64 dask.array<chunksize=(125, 3), meta=np.ndarray>
    Factory         (i, AOP) float64 dask.array<chunksize=(125, 3), meta=np.ndarray>
    Logistics       (i, AOP) float64 dask.array<chunksize=(125, 3), meta=np.ndarray>
    ...              ...
    Fuel            (i, AOP) float64 dask.array<chunksize=(125, 3), meta=np.ndarray>
    Recycling       (i, AOP) float64 dask.array<chunksize=(125, 3), meta=np.ndarray>
    Incineration    (i, AOP) float64 dask.array<chunksize=(125, 3), meta=np.ndarray>
    Landfill        (i, AOP) float64 dask.array<chunksize=(125, 3), meta=np.ndarray>
    Prototypes      (i, AOP) float64 dask.array<chunksize=(125, 3), meta=np.ndarray>
    Certification   (i, AOP) float64 dask.array<chunksize=(125, 3), meta=np.ndarray>
Attributes:
    Name:     Antera_spray

In [21]:
aeco.mean("EP", by="sum")

<xarray.DataArray 'stack-b5b16b130f34ae7cec71c9f1bb37b2aa' (AOP: 3)>
dask.array<mean_agg-aggregate, shape=(3,), dtype=float64, chunksize=(3,), chunktype=numpy.ndarray>
Coordinates:
  * AOP      (AOP) <U2 'HH' 'ED' 'RA'

## Contribution to Variance (CTV)

Calculate CTV results for all inputs with midpoint and endpoint results. CTV results can be acessed by the `LCIA.CTV` attribute.

In [22]:
aeco.build_CTV(parameterset=p)

<xarray.Dataset>
Dimensions:     (Parameters: 84)
Coordinates:
  * Parameters  (Parameters) <U18 'takt_US' 'takt_BR' ... 'p_recycl_CFRP'
Data variables: (12/21)
    FE          (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    MRD         (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    POF         (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    MET         (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    TA          (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    ULO         (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    ...          ...
    FD          (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    WD          (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    CC          (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    HH          (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    ED          (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    RA          (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>

In [49]:
aeco.CTV

c:\Users\giparoli\Documents\Projetos\AEco\Scripts\Tools_Antera.py:309: RuntimeWarning: invalid value encountered in true_divide
  return covariance_gufunc(x, y) / (x.std(axis=-1) * y.std(axis=-1))


<xarray.Dataset>
Dimensions:     (Parameters: 84)
Coordinates:
  * Parameters  (Parameters) <U18 'takt_US' 'takt_BR' ... 'p_recycl_CFRP'
Data variables: (12/21)
    FE          (Parameters) float64 0.02222 0.002931 0.0 ... 0.0 0.0 0.0
    MRD         (Parameters) float64 0.0313 0.04507 0.0 2.704 ... 0.0 0.0 0.0
    POF         (Parameters) float64 0.04116 0.1082 0.0 0.0243 ... 0.0 0.0 0.0
    MET         (Parameters) float64 0.09443 0.009022 0.0 ... 0.0 0.0 0.0
    TA          (Parameters) float64 0.03795 0.09158 0.0 0.0283 ... 0.0 0.0 0.0
    ULO         (Parameters) float64 0.02976 0.07275 0.0 0.02436 ... 0.0 0.0 0.0
    ...          ...
    FD          (Parameters) float64 0.04265 0.1146 0.0 0.02493 ... 0.0 0.0 0.0
    WD          (Parameters) float64 0.03949 0.09522 0.0 0.02759 ... 0.0 0.0 0.0
    CC          (Parameters) float64 0.04298 0.1149 0.0 0.02006 ... 0.0 0.0 0.0
    HH          (Parameters) float64 0.002447 0.1957 0.0 6.089 ... 0.0 0.0 0.0
    ED          (Parameters) float64 0.04313 0.1182 0.0 0.01176 ... 0.0 0.0 0.0
    RA          (Parameters) float64 0.04179 0.114 0.0 0.01739 ... 0.0 0.0 0.0

In [23]:
aeco.CTV["CC"].sortby(aeco.CTV["CC"], ascending=False)

c:\Users\giparoli\Documents\Projetos\AEco\Scripts\Tools_Antera.py:309: RuntimeWarning: invalid value encountered in true_divide
  return covariance_gufunc(x, y) / (x.std(axis=-1) * y.std(axis=-1))


<xarray.DataArray 'CC' (Parameters: 84)>
dask.array<getitem, shape=(84,), dtype=float64, chunksize=(1,), chunktype=numpy.ndarray>
Coordinates:
  * Parameters  (Parameters) <U18 'FH' 't_cruise' ... 'productivity' 'OEW'

# Export Results

Export MP, EP, LCI and CTV result to NetCDF file.

In [56]:
aeco.save(output_path, LCI=False)

In [22]:
#aeco.to_excel(output_path)